In [1]:
#lstm Multivariate MultiStep Example OVer Air Pollution Dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,LSTM,Activation, Input

from sklearn.preprocessing import LabelEncoder

#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf

#Before do anything else do not forget to reset the backend for the next iteration (rerun the model)
keras.backend.clear_session()

In [2]:
dataset = pd.read_csv("input/lstm-datasets-multivariate-univariate/LSTM-Multivariate_pollution.csv", header=0, index_col=0)

t = dataset.columns.tolist()

dataset = dataset[['dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain','pollution']]

In [3]:

#else slice is invalid for use in labelEncoder
dataset= dataset.values
# integer encode direction
encoder = LabelEncoder()
dataset[:,3] = encoder.fit_transform(dataset[:,3])

#conver to pd.Dataframe else slices error

dataset = pd.DataFrame(dataset)
dataset.columns = ['dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain','pollution']

######## ensure all data is float

In [4]:
#dataset[['dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain','pollution']]
#Data Pre-processing step--------------------------------
x_1 = dataset['dew'].values
x_2 = dataset['temp'].values
x_3 = dataset['press'].values
x_4 = dataset['wnd_spd'].values
x_5 = dataset['wnd_dir'].values
x_6 = dataset['snow'].values
x_7 = dataset['rain'].values
y = dataset['pollution'].values
#x_1 = x_1.values
#x_2 = x_2.values
#y = y.values


# Step 1 : convert to [rows, columns] structure
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_3.reshape((len(x_3), 1))
x_4 = x_4.reshape((len(x_4), 1))
x_5 = x_5.reshape((len(x_5), 1))
x_6 = x_6.reshape((len(x_6), 1))
x_7 = x_7.reshape((len(x_7), 1))
y = y.reshape((len(y), 1))
print ("x_1.shape" , x_1.shape) 
print ("x_2.shape" , x_2.shape) 
print ("y.shape" , y.shape)

# Step 2 : normalization 
scaler_x_1 = MinMaxScaler(feature_range=(0, 1)).fit(x_1)
x_1_scaled = scaler_x_1.transform(x_1)
scaler_x_2 = MinMaxScaler(feature_range=(0, 1)).fit(x_2)
x_2_scaled = scaler_x_2.transform(x_2)
scaler_x_3 = MinMaxScaler(feature_range=(0, 1)).fit(x_3)
x_3_scaled = scaler_x_3.transform(x_3)
scaler_x_4 = MinMaxScaler(feature_range=(0, 1)).fit(x_4)
x_4_scaled = scaler_x_4.transform(x_4)
scaler_x_5 = MinMaxScaler(feature_range=(0, 1)).fit(x_5)
x_5_scaled = scaler_x_5.transform(x_5)
scaler_x_6 = MinMaxScaler(feature_range=(0, 1)).fit(x_6)
x_6_scaled = scaler_x_6.transform(x_6)
scaler_x_7 = MinMaxScaler(feature_range=(0, 1)).fit(x_7)
x_7_scaled = scaler_x_7.transform(x_7)
scaler_y = MinMaxScaler(feature_range=(0, 1)).fit(y)
y_scaled = scaler_y.transform(y)

# Step 3 : horizontally stack columns
dataset_stacked = hstack((x_1_scaled, x_2_scaled,x_2_scaled, x_3_scaled,
                          x_4_scaled, x_5_scaled,x_7_scaled, y_scaled))
print ("dataset_stacked.shape" , dataset_stacked.shape)

x_1.shape (43800, 1)
x_2.shape (43800, 1)
y.shape (43800, 1)
dataset_stacked.shape (43800, 8)


In [5]:
#1. n_steps_in : Specify how much data we want to look back for prediction
#2. n_step_out : Specify how much multi-step data we want to forecast

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
 X, y = list(), list()
 for i in range(len(sequences)):
  # find the end of this pattern
  end_ix = i + n_steps_in
  out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
  if out_end_ix > len(sequences):
   break
  # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

# choose a number of time steps #change this accordingly
n_steps_in, n_steps_out = 60 , 1#30
# covert into input/output
X, y = split_sequences(dataset_stacked, n_steps_in, n_steps_out)
print ("X.shape" , X.shape) 
print ("y.shape" , y.shape)

X.shape (43741, 60, 7)
y.shape (43741, 1)


In [6]:
#spliting the dataset--------------------------

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
train_X, test_X,train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0)

train_X.shape
train_y.shape 
test_X.shape 
test_y.shape 
n_features = 7
batch_size=64

In [12]:
train_X.shape


(34992, 60, 7)

In [7]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
# model = Sequential()
# model.add(LSTM(50, activation='relu', stateful=True, return_sequences=True, input_shape=(n_steps_in, n_features)))
# model.add(LSTM(50, activation='relu', stateful=True, return_sequences=True, input_shape=(n_steps_in, n_features)))
# model.add(Dense(n_steps_out))
# model.add(Activation('linear'))
# model.compile(loss='mse' , optimizer=opt , metrics=['accuracy'])

input_1 = Input(batch_shape=(batch_size, n_steps_in, n_features))
lstm_layer_1 = LSTM(10, stateful=True, return_sequences=True)(input_1)
lstm_layer_2 = LSTM(10, stateful=True, return_sequences=True)(lstm_layer_1)
output_1 = Dense(units=1)(lstm_layer_2)

wildwilly=Model(inputs=input_1, outputs=output_1)
wildwilly.summary()
wildwilly.compile(loss='mse' , optimizer=opt , metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 60, 7)]             0         
_________________________________________________________________
lstm (LSTM)                  (64, 60, 10)              720       
_________________________________________________________________
lstm_1 (LSTM)                (64, 60, 10)              840       
_________________________________________________________________
dense (Dense)                (64, 60, 1)               11        
Total params: 1,571
Trainable params: 1,571
Non-trainable params: 0
_________________________________________________________________


In [9]:

# Fit network #increase the epochs for better model training
# history = model.fit(train_X , train_y , epochs=50, steps_per_epoch=25 , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False)
history = wildwilly.fit(train_X , train_y , epochs=50, batch_size=batch_size , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False)


Epoch 1/50
543/547 [============================>.] - ETA: 0s - loss: 0.0094 - accuracy: 0.0469

InvalidArgumentError:    Specified a list with shape [64,7] from a tensor with shape [48,7]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[model/lstm/PartitionedCall]] [Op:__inference_train_function_4465]

Function call stack:
train_function -> train_function -> train_function


In [ ]:

#TEST DATA----------------------------------------

dataset_test_ok = pd.read_csv('input/lstm-datasets-multivariate-univariate/pollution_test_data1.csv')
dataset_test_ok.head()

# integer encode direction
encoder1 = LabelEncoder()
dataset_test_ok.iloc[:,3] = encoder1.fit_transform(dataset_test_ok.iloc[:,3])


# read test data
x1_test = dataset_test_ok['dew'].values
x2_test = dataset_test_ok['temp'].values
x3_test = dataset_test_ok['press'].values
x4_test = dataset_test_ok['wnd_spd'].values
x5_test = dataset_test_ok['wnd_dir'].values
x6_test = dataset_test_ok['snow'].values
x7_test = dataset_test_ok['rain'].values
y_test = dataset_test_ok['pollution'].values # no need to scale


# convert to [rows, columns] structure
x1_test = x1_test.reshape((len(x1_test), 1))
x2_test = x2_test.reshape((len(x2_test), 1))
x3_test = x3_test.reshape((len(x3_test), 1))
x4_test = x4_test.reshape((len(x4_test), 1))
x5_test = x5_test.reshape((len(x5_test), 1))
x6_test = x6_test.reshape((len(x6_test), 1))
x7_test = x7_test.reshape((len(x7_test), 1))
y_test = y_test.reshape((len(y_test), 1))


x1_test_scaled = scaler_x_1.transform(x1_test)
x2_test_scaled = scaler_x_2.transform(x2_test)
x3_test_scaled = scaler_x_3.transform(x3_test)
x4_test_scaled = scaler_x_4.transform(x4_test)
x5_test_scaled = scaler_x_5.transform(x5_test)
x6_test_scaled = scaler_x_6.transform(x6_test)
x7_test_scaled = scaler_x_7.transform(x7_test)


# Step 3 : horizontally stack columns
dataset_test_stacked = hstack((x1_test_scaled,x2_test_scaled,x3_test_scaled,x4_test_scaled,
                               x5_test_scaled,x6_test_scaled,x7_test_scaled))


print ("dataset_stacked.shape" , dataset_test_stacked.shape)

In [ ]:
###Prediction#######################################################

dataset_test_X = dataset_test_stacked
print("dataset_test_X :",dataset_test_X.shape)
test_X_new = dataset_test_X.reshape(1,dataset_test_X.shape[0],dataset_test_X.shape[1])

y_pred = model.predict(test_X_new)

y_pred_inv = scaler_y.inverse_transform(y_pred)
y_pred_inv = y_pred_inv.reshape(n_steps_out,1)
y_pred_inv = y_pred_inv[:,0]
print("y_pred :",y_pred.shape)
print("y_pred_inv :",y_pred_inv.shape)

In [ ]:
scaler_y.inverse_transform(y_pred)

In [ ]:
y_test

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(y_test)
plt.plot(y_pred_inv)
plt.show()